# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique:
* Model:
* Evaluation approach:
* Fine-tuning dataset:

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

MODEL_NAME = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

dataset = load_dataset("rotten_tomatoes")

test_data = dataset["test"].shuffle(seed=42).select(range(500))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

: 

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_test_data = test_data.map(tokenize_function, batched=True)

tokenized_test_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

: 

In [ ]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(tokenized_test_data, batch_size=8)

def evaluate_model(model, dataloader):
    model.eval()
    total, correct = 0, 0

    with torch.no_grad():
        for batch in dataloader:
            inputs = {key: batch[key].to(model.device) for key in ["input_ids", "attention_mask"]}
            labels = batch["label"].to(model.device)

            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=-1)

            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    return correct / total

: 

In [ ]:
accuracy = evaluate_model(model, test_dataloader)
print(f"Pre-trained Model Accuracy on Test Set: {accuracy:.2%}")

: 

## Performing Parameter-Efficient Fine-Tuning

1.   List item
2.   List item



TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"]
)

peft_model = get_peft_model(model, lora_config)
peft_model.to(device)

peft_model.print_trainable_parameters()

: 

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_train_data = tokenized_dataset["train"]
tokenized_test_data = tokenized_dataset["test"]

tokenized_train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
tokenized_test_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

: 

In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np

training_args = TrainingArguments(
    output_dir="/tmp/results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="/tmp/logs",
    logging_steps=10,
    save_total_limit=2,
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_test_data,
    tokenizer=tokenizer,
    data_collator=None,
    compute_metrics=compute_metrics,
)

: 

In [ ]:
trainer.train()

: 

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [ ]:
# Saving the model
# model.save("/tmp/your_model_name")
peft_model.save_pretrained("/tmp/peft_model")
tokenizer.save_pretrained("/tmp/peft_model")

: 

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
from transformers import AutoModelForSequenceClassification

peft_model = AutoModelForSequenceClassification.from_pretrained("/tmp/peft_model")
peft_model.to(device)

peft_model_accuracy = evaluate_model(peft_model, test_dataloader)

print(f"PEFT Fine-Tuned Model Accuracy: {peft_model_accuracy:.2%}")

: 

In [ ]:
print(f"Base Model Accuracy: {accuracy:.2%}")
print(f"PEFT Model Accuracy: {peft_model_accuracy:.2%}")

: 

: 

: 

: 